In [ ]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data#python-sdk


In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [2]:
from azureml.core.datastore import Datastore

In [5]:
batchscore_blob = Datastore.register_azure_blob_container(ws, 
                      datastore_name="images_datastore", 
                      container_name="sampledata", 
                      account_name="pipelinedata", 
                      overwrite=True)
def_data_store = ws.get_default_datastore()

In [7]:
from azureml.core.dataset import Dataset
from azureml.pipeline.core import PipelineData

In [8]:
input_images = Dataset.File.from_files((batchscore_blob, "batchscoring/images/"))
label_ds = Dataset.File.from_files((batchscore_blob, "batchscoring/labels/"))
output_dir = PipelineData(name="scores", 
                          datastore=def_data_store, 
                          output_path_on_compute="batchscoring/results")

In [9]:
def_data_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-2dd703c0-9f8b-4e31-bb4d-9ff3f50aa329",
  "account_name": "amlworkbook2803154736",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [14]:
input_images = input_images.register(workspace = ws, name = "input_images")
label_ds = label_ds.register(workspace = ws, name = "label_ds")

In [15]:
import os
import tarfile
import urllib.request

if not os.path.isdir("models"):
    os.mkdir("models")
    
response = urllib.request.urlretrieve("http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz", "model.tar.gz")
tar = tarfile.open("model.tar.gz", "r:gz")
tar.extractall("models")